### Import lib

In [1]:
import os
os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from treeconfigparser import TreeConfigParser
import joblib

import models.reg as rg
import models.seg as sg
from data.Data import Data
from include.loss import pixel_weighted_cross_entropy

Segmentation Models: using `keras` framework.


## Download func

In [2]:
path_res = "/cerea_raid/users/dumontj/dev/coco2/dl/res"

## CLEAN POUR BIEN PIOCHER VALIDATION DATASET : BIZARRE ICI

In [3]:
def get_data_for_reg(ds: str, scaler):
    config = TreeConfigParser()
    config.readfiles(os.path.join(path_res, ds, "config.cfg"))
    shuffle_indices = np.fromfile(os.path.join(path_res, ds, "shuffle_indices.bin")).astype(np.int64)
    data = Data(config, shuffle_indices)
    data.prepare_input(scaler)
    data.prepare_output()
    print("xco2 valid mean", np.mean(data.y.valid[0]))
    return data

In [4]:
def get_model(path_w: str):
    model = keras.models.load_model(os.path.join(path_w, "weights_cp_best.h5"), compile=True)
    return model

def get_scaler(path_w: str):
    scaler = joblib.load(os.path.join(path_w, "scaler.save"))
    return scaler

In [5]:
def eval_ds_with_model(data, model):
    x_valid = tf.convert_to_tensor(data.x.valid[0], np.float32)
    y_valid = tf.convert_to_tensor(data.y.valid, np.float32)
    loss_valid = model.evaluate(x=x_valid, y=y_valid, batch_size=64)
    return loss_valid

In [6]:
def eval_ds_with_baseline(data, model):
    y_train = tf.convert_to_tensor(data.y.train, np.float32)
    mean_train = tf.math.reduce_mean(y_train) * tf.ones_like(y_train, np.float32)
    loss_baseline = model.compute_loss(y=y_train, y_pred=mean_train).numpy()
    return loss_baseline

### Case choices

### REG basic

In [7]:
names_ds = ["par", "ber", "pw"]
df_indices_ds = ["ds_" + case for case in names_ds]
keys_ds = ["reg_" + case for case in names_ds]

names_w = ["par", "ber", "pw", "ber_DA"]
df_indices_w = ["w_" + case for case in names_w]
keys_w = ["reg_" + case for case in names_w]

### REG advanced

In [14]:
names_ds = ["ber", "PGIPW", "PGIPP_DA"]
df_indices_ds = ["ds_" + case for case in names_ds]
keys_ds = ["reg_" + case for case in names_ds]

names_w = ["ber", "ber_DA", "PGIPW", "PGIPW_DA", "PGIPW_DA2", "PGIPP_DA", "PGIPW_DA_save"]
df_indices_w = ["w_" + case for case in names_w]
keys_w = ["reg_" + case for case in names_w]

### Make df of results

In [ ]:
df = pd.DataFrame(dict(zip(df_indices_w, [[-1.]*len(names_ds)]*len(names_w))))
df.index = df_indices_ds
for df_idx_w, key_w in zip(df_indices_w, keys_w):
    full_path_w = os.path.join("/cerea_raid/users/dumontj/dev/coco2/dl/res", key_w)
    model = get_model(full_path_w)
    scaler = get_scaler(full_path_w)
    for df_idx_ds, key_ds in zip(df_indices_ds, keys_ds):
        data = get_data_for_reg(key_ds, scaler)
        loss_valid = eval_ds_with_model(data, model)
        df.loc[df_idx_ds, df_idx_w] = loss_valid  

In [16]:
df

,w_ber,w_ber_DA,w_PGIPW,w_PGIPW_DA,w_PGIPW_DA2,w_PGIPP_DA,w_PGIPW_DA_save
ds_ber,0.058189,0.062930,0.215092,0.191444,0.161283,0.200780,-1.0
ds_PGIPW,0.295638,0.298279,0.082520,0.047482,0.070796,0.047488,-1.0
ds_PGIPP_DA,0.297590,0.295858,0.075448,0.045666,0.071509,0.053917,-1.0


In [14]:
df

,w_ber,w_ber_DA,w_PGIPW,w_PGIPW_DA,w_PGIPW_DA2
ds_ber,0.164924,0.059262,0.211345,0.189556,0.157331
ds_PGIPW,0.298534,0.295204,0.080281,0.039498,0.071498


#### Add baseline

In [15]:
df["mean_bl"] = [-1.]*len(df.index)
for df_idx_ds, key_ds in zip(df_indices_ds, keys_ds):
    config_file = os.path.join("/cerea_raid/users/dumontj/dev/coco2/dl/cfg", key_ds + ".cfg")
    config = TreeConfigParser()
    config.readfiles(config_file)
    data = Data(config)
    data.prepare_output()
    
    mean_bl = eval_ds_with_baseline(data, model)
    df.loc[df_idx_ds, "mean_bl"] = mean_bl

data.out.train.shape (6874, 1)
data.out.train.shape (24194, 1)


In [16]:
df

,w_ber,w_ber_DA,w_PGIPW,w_PGIPW_DA,w_PGIPW_DA2,mean_bl
ds_ber,0.164924,0.059262,0.211345,0.189556,0.157331,0.185153
ds_PGIPW,0.298534,0.295204,0.080281,0.039498,0.071498,0.189505
